In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [2]:
# patient list

patient_df = pd.read_excel('/Users/yena/Desktop/SCH/척척석사/EMR related/낙상데이터/천안/(천안병원)간호간병통합병동 낙상대상자 리스트(2017-2021.10).xlsx', index_col =0)
patient_list = patient_df['등록번호'].astype(str).unique()  # 등록번호 unique -> 228명 확인
fall_date = patient_df[['등록번호', '낙상발생일시']]

duration=[]

for i in fall_date['낙상발생일시'].tolist():
    i = str(i)
    i = i.split(' ')[0]
    duration.append(i)

fall_date['낙상발생일시'] = duration



In [3]:
# str_to_dt 함수

import datetime

def str_to_dt(x):
    x = str(x)
    d = int(x[0:4])
    y = int(x[4:6])
    z = int(x[6:8])
    # hours = int(x[8:10])
    # mins = int(x[10:12])
    # secs = int(x[12:])
    # str_data = (d+','+y+','+z+','+hours+','+mins+','+secs)
    new_time = datetime.datetime(d,y,z)
    new_time_no = new_time.replace(microsecond = 0)
    
    return new_time_no

In [4]:
needs = pd.read_csv('/Users/yena/Desktop/SCH/척척석사/EMR related/낙상데이터/천안/간호필요도_17_21.csv')

In [5]:
needs.head(3)

,Unnamed: 0,입원일자,등록번호,환자명,도구명,상세내역명,체크,평가일자,평가시간,도구구분,성별,나이,진료과,병동
0,0,20170625.0,57121,김문오,ADL_체위변경,도움없음(침대에서 뒤척임을 환자 스스로 할 수 있는 경우),Y,20170701,800,간호필요도,M,64,MH,BA03B
1,1,20170625.0,57121,김문오,전문치료 시행,"7개중 1개 이상/1일(마약성주사, 항암치료, 승압제, 항부정맥제, 면역억제제, 항...",Y,20170701,800,간호필요도,M,64,MH,BA03B
2,2,20170625.0,57121,김문오,ADL_식사섭취,"일부도움(필요에 따라 식사 섭취 행위의 일부에 도움을 주는 경우, 식탁에서 먹기 편...",Y,20170701,800,간호필요도,M,64,MH,BA03B


In [6]:
len(needs['등록번호'].unique())

12052

In [7]:
needs = needs[['등록번호', '도구명', '상세내역명', '평가일자']]
needs['등록번호'] = needs['등록번호'].astype(str)
needs = needs[needs['등록번호'].isin(set(patient_list))]
needs['평가일자'] = needs['평가일자'].apply(str_to_dt)
print(needs.shape)
print(len(needs['등록번호'].unique()))
needs.head(3)

(67922, 4)
70


,등록번호,도구명,상세내역명,평가일자
465,1393198,V/S측정,1~6회이상/1일,2017-07-01
466,1393198,병실종류,6인실,2017-07-01
467,1393198,수술여부,수술미실시,2017-07-01


In [8]:
needs['도구명'] = needs['도구명'].replace({'ADL_배변':'ADL_배뇨, 배변'})
needs['도구명'] = needs['도구명'].replace({'흡인간호(기관 내)':'호흡 간호_비침습적 산소 투여'})
needs['도구명'] = needs['도구명'].replace({'ADL_이동':'ADL_침상 밖으로 이동'})

tools=['ADL_배뇨, 배변','호흡 간호_비침습적 산소 투여', 'ADL_체위변경', 'ADL_침상 밖으로 이동', 'ADL_식사섭취', '위험행동관리', '전문치료 시행']
needs = needs[needs['도구명'].isin(set(tools))]

needs= needs.drop_duplicates()
print(needs.shape)
print(len(needs['등록번호'].unique()))

(20268, 4)
70


In [9]:
fall_date = patient_df[['등록번호', '낙상발생일시']]   

In [10]:
fall_date['등록번호'] = fall_date['등록번호'].astype(str)
needs = needs.merge(fall_date, on=['등록번호'], how='inner')
print(len(needs['등록번호'].unique()))

70


In [11]:
needs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21624 entries, 0 to 21623
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   등록번호    21624 non-null  object        
 1   도구명     21624 non-null  object        
 2   상세내역명   21624 non-null  object        
 3   평가일자    21624 non-null  datetime64[ns]
 4   낙상발생일시  21624 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 1013.6+ KB


In [12]:
def str_clean(x):
    x = str(x)
    x = x.replace('-','')
    x = str_to_dt(x)

    return x

In [13]:
needs['낙상발생일시'] = needs['낙상발생일시'].apply(str_clean)
needs['평가일자'] = needs['평가일자'].apply(str_clean)

In [14]:
needs['유무'] = needs['낙상발생일시']-needs['평가일자']
needs['유무'] = needs['유무'].astype(str)
cond = ['0 days', '1 days', '2 days', '3 days']
needs = needs[needs['유무'].isin(set(cond))]
print(len(needs['등록번호'].unique()))

60


In [15]:
needs

,등록번호,도구명,상세내역명,평가일자,낙상발생일시,유무
0,1393198,"ADL_배뇨, 배변",도움없음(배변을 위한 일련의 행위 모두를 스스로 할 수 있는 경우),2017-07-01,2017-07-02,1 days
3,1393198,ADL_식사섭취,"일부도움(필요에 따라 식사 섭취 행위의 일부에 도움을 주는 경우, 식탁에서 먹기 편...",2017-07-01,2017-07-02,1 days
6,1393198,ADL_침상 밖으로 이동,"도움없음(도움없이 이동할 수 있는 경우, 기어서라도 이동을 스스로 할 수 있는경우도...",2017-07-01,2017-07-02,1 days
9,1393198,ADL_체위변경,도움없음(침대에서 뒤척임을 환자 스스로 할 수 있는 경우),2017-07-01,2017-07-02,1 days
12,1393198,전문치료 시행,"7개중 1개 이상/1일(마약성주사, 항암치료, 승압제, 항부정맥제, 면역억제제, 항...",2017-07-01,2017-07-02,1 days
...,...,...,...,...,...,...
21607,860045,ADL_식사섭취,"도움 없음(도움, 관찰 없이 스스로 식사 가능한 경우, 보조기구 이용 스스로 식사 ...",2021-07-25,2021-07-25,0 days
21608,860045,"ADL_배뇨, 배변","일부 도움(도움 없이 배변, 배뇨를 위한 일련의 행위를 하는데 어려움이 있어 간호직...",2021-07-25,2021-07-25,0 days
21609,860045,호흡 간호_비침습적 산소 투여,해당없음(ex.비침습적 양압환기 및 가온 가습 고유량 비강캐뉼라 요법),2021-07-25,2021-07-25,0 days
21610,860045,ADL_체위변경,"도움없음(도움 없이 스스로 변경 가능, 한쪽으로만 변경이 가능해도 됨, 침대 난간,...",2021-07-25,2021-07-25,0 days


In [16]:
needs['유무'].value_counts()

0 days    378
1 days    354
2 days    300
3 days    270
Name: 유무, dtype: int64

In [17]:
new_danger = []
new_tools = []
for num, i in enumerate(needs['도구명'].tolist()):
    new_tools.append(i)
    if needs['도구명'].tolist()[num]=='위험행동관리':
        detail_str = '위험 '+needs['상세내역명'].tolist()[num]
        new_danger.append(detail_str)
    else:
        new_danger.append(needs['상세내역명'].tolist()[num])

print(len(new_danger))
print(len(new_tools))

needs['도구명'] = new_tools
needs['상세내역명'] = new_danger

1302
1302


In [18]:
test = needs.copy()
test['agg'] = test['등록번호'].astype(str)+'_'+ test['평가일자'].astype(str)+'_'+test['상세내역명']

In [19]:
from tqdm import tqdm   # 작업 진행률 표시

ADL_uri = []
ADL_resp = []
ADL_pos = []
ADL_outbed =[]
ADL_meal=[]

ADL_spe = [] ## 전문치료실행
ADL_danger = []

# ADL_oral_drug = []
# ADL_immune = []
# ADL_pres = [] ##승압제
# ADL_antipres = [] ##항부정
# ADL_oral_can = [] ## 항암
# ADL_cancer = [] ##항암주사
# ADL_clog = [] ## 혈전



new_patient_list = []
new_date_list = []

for num, i in tqdm(enumerate(test['agg'].tolist())):
    patient = i.split('_')[0]
    new_patient_list.append(patient)
    date = i.split('_')[1]
    new_date_list.append(date)
    detail = i.split('_')[2]
  
    ## 배뇨배변 ###
    if detail =="도움 없음(도움 없이 배변, 배뇨를 위한 일련의 행위, 장루관리를 스스로 할 수 있는 경우, 관장하는 경우)":
        detail_input = patient+'_'+date+'_'+'0'
        ADL_uri.append(detail_input)
    elif detail =="도움없음(배변을 위한 일련의 행위 모두를 스스로 할 수 있는 경우)":
        detail_input = patient+'_'+date+'_'+'0'
        ADL_uri.append(detail_input)
    elif detail == '일부 도움(도움 없이 배변, 배뇨를 위한 일련의 행위를 하는데 어려움이 있어 간호직원 등의 일부 도움이 반드시 필요한 경우,  정서장애, 인지장애, 혈압조절장애 등으로 인하여 배변, 배뇨를 하는 동안 반드시 누군가 곁에서 관찰, 감독이 필요한 경우, Foley-catheter 적용중인 경우)(nelaton catheter제외)':
        detail_input = patient+'_'+date+'_'+'1'
        ADL_uri.append(detail_input)
    elif detail == '일부도움(배변을 위한 일련의 행위에 일부 도움이 행해지는 경우)':
        detail_input = patient+'_'+date+'_'+'1'
        ADL_uri.append(detail_input)
    elif detail =='전부 도움(스스로 배변 배뇨를 위한 일련의 행위 모두를 전혀 할 수 없어 간호 직원 등에 의해 전적인 도움 필요, 기저귀 착용,장루관리를 간호직원이 하는 경우)':
        detail_input = patient+'_'+date+'_'+'2'
        ADL_uri.append(detail_input)
    elif detail =='전부도움(배변을 위한 일련의 행위에 전부 도움이 행해지는 경우, 활동에 제한이 있어 기저귀를 적용한 환자 모두 포함)':
        detail_input = patient+'_'+date+'_'+'2'
        ADL_uri.append(detail_input)

    ### 식사섭취###
    if detail == '도움 없음(도움, 관찰 없이 스스로 식사 가능한 경우, 보조기구 이용 스스로 식사 가능한 경우, NPO, TPN 등의 사유로 식사하는 행위가 미발생 하는 경우)':
        detail_input = patient+'_'+date+'_'+'0'
        ADL_meal.append(detail_input)
    elif detail == '도움없음(도움, 지켜보는 것 없이 스스로 식사를 할 수 있는 경우, 젓가락, 숟가락 이외의 다른 보조기구등을 사용하는 경우, 식사중지, 단식의 경우)':
        detail_input = patient+'_'+date+'_'+'0'
        ADL_meal.append(detail_input)
    elif detail == '일부 도움(간단한 식사도구는 스스로 다룰 수 있으나 작게 자르기, 껍질 벗기기, 생선 바르기 등을 스스로 할 수 없어 간호직원 등의 도움이 필요한 경우,  흡인( aspiration) 과거력, 인지장애, 연하곤란(dysphagia) 등으로 인하여 식사하는 동안 반드시 누군가 곁에서 관찰, 감독이 필요한 경우) ':
        detail_input = patient+'_'+date+'_'+'1'
        ADL_meal.append(detail_input)
    elif detail == '일부도움(필요에 따라 식사 섭취 행위의 일부에 도움을 주는 경우, 식탁에서 먹기 편하도록 배려하는 행위가 이루어 지고 있는 경우, 식사 중 한 가지라도 도움을 주는 경우, 지켜보며 지시가 필요한 경우)':
        detail_input = patient+'_'+date+'_'+'1'
        ADL_meal.append(detail_input)
    elif detail == '전부 도움(혼자서는 식사를 전혀 할 수 없어 간호직원 등에 의해 모든 과정에서 전적인 도움(스푼 피딩)을 받아야 함)':
        detail_input = patient+'_'+date+'_'+'2'
        ADL_meal.append(detail_input)
    elif detail == '전부도움(스스로는 전혀 먹을 수 없고 모든 것에 도움이 필요한 경우, 식사 시작부터 끝까지 모든 도움이 필요한 경우)':
        detail_input = patient+'_'+date+'_'+'2'
        ADL_meal.append(detail_input)
         

    ### 체위변경 ###
    if detail =='도움없음(도움 없이 스스로 변경 가능, 한쪽으로만 변경이 가능해도 됨, 침대 난간, 끈, 바, 사이드 레일 등을 사용하는 경우)':
        detail_input = patient+'_'+date+'_'+'0'
        ADL_pos.append(detail_input)
    elif detail =='도움없음(침대에서 뒤척임을 환자 스스로 할 수 있는 경우)':
        detail_input = patient+'_'+date+'_'+'0'
        ADL_pos.append(detail_input)
    elif detail =='일부 도움(스스로 체위 변경을 하는데 어려움이 있어 간호 직원 등이 팔 또는 몸의 일부를 지지하는 도움이 반드시 필요, 환자가 스스로 할 수 있으나, 간호직원 등이 도움을 주는 경우 제외)':
        detail_input = patient+'_'+date+'_'+'1'
        ADL_pos.append(detail_input)
    elif detail =='일부도움(침대 난간, 끈, 바, 사이드레일 등을 잡으면 혼자서 침대에서 뒤척임을 할 수 있는 경우)':
        detail_input = patient+'_'+date+'_'+'1'
        ADL_pos.append(detail_input)
    elif detail =="전부 도움(체위 변경을 혼자서는 전혀 할 수 없어 간호 직원 등에 의해 모든 과정에서 전적인 도움을 받아야 함)":
        detail_input = patient+'_'+date+'_'+'2'
        ADL_pos.append(detail_input)
    elif detail =="전부도움(침대에서 뒤척이는데 도움이 필요한 경우)":
        detail_input = patient+'_'+date+'_'+'2'
        ADL_pos.append(detail_input)

    ### 호흡 ###
    if detail =='해당없음(ex.비침습적 양압환기 및 가온 가습 고유량 비강캐뉼라 요법)':
        detail_input = patient+'_'+date+'_'+'0'
        ADL_resp.append(detail_input)
    elif detail =='해당없음':
        detail_input = patient+'_'+date+'_'+'0'
        ADL_resp.append(detail_input)
    elif detail =='비강캐뉼라, 산소마스크 등을 이용하여 산소투여하고 시행하고 기록이 있는 경우':
        detail_input = patient+'_'+date+'_'+'1'
        ADL_resp.append(detail_input)
    elif detail =='T-cannula, E-tube 등을 통한 흡인간호, 드레싱 교환, T-cannula 교환':
        detail_input = patient+'_'+date+'_'+'1'
        ADL_resp.append(detail_input)
    elif detail =='T-cannula, E-tube 등 을 통한 인공호흡기 사용, NPPV 또는 가온 가습 고유량 비강 캐뉼라 요법(optiflow)을 실시하고 기록이 있는 경우':
        detail_input = patient+'_'+date+'_'+'1'
        ADL_resp.append(detail_input)

    ### 침상이동 ###
    if detail == '도움 없음(도움 없이 스스로 침상 밖으로 이동이 가능, 무언가 잡거나 보조기구를 이용하여 스스로 가능한 경우, 와상, 절대안정 등의 사유로 침상 밖으로 이동이 없는 경우에 해당, 이동하기 위해 보조기구를 준비, 사이드 레일을 내리는 등의 도움은 포함하지 않음)':
        detail_input = patient+'_'+date+'_'+'0'
        ADL_outbed.append(detail_input)
    elif detail == '도움없음(도움없이 이동할 수 있는 경우, 기어서라도 이동을 스스로 할 수 있는경우도 포함)':
        detail_input = patient+'_'+date+'_'+'0'
        ADL_outbed.append(detail_input)
    elif detail == '일부 도움(스스로 침상 밖으로 나오는데 어려움이 있어 팔 또는 몸의 일부를 지지하는 도움이나 관찰이  반드시 필요함, 환자의 허약, 장애, 약물, 낙상 과거력, 치료기구 등으로 인하여 침상 밖으로 이동할 때마다 관찰이 필요한 경우)':
        detail_input = patient+'_'+date+'_'+'1'
        ADL_outbed.append(detail_input)
    elif detail == '일부도움(직접 도움을 줄 필요는 없지만 사고 등이 나지 않도록 지켜보는 경우, 들것에 이동 시 환자가 자력으로 조금씩 이동할 수 있는 경우, 부분적 도움이 행해지는 경우, 위험이 없도록 간호사 등이 추가 도움을 주는 경우)':
        detail_input = patient+'_'+date+'_'+'1'
        ADL_outbed.append(detail_input)
    elif detail == '전부 도움(혼자서는 침상 밖으로 이동을 전혀 할 수 없어 간호직원 등에 의해 안고, 옮겨주는(기구 혹은 장비 이용 포함) 등의  전적인 도움을 받는 경우)':
        detail_input = patient+'_'+date+'_'+'2'
        ADL_outbed.append(detail_input)
    elif detail == '전부도움(자신이 이동을 전혀 할 수 없는 경우, 전면적인 도움이 행해지는 경우)':
        detail_input = patient+'_'+date+'_'+'2'
        ADL_outbed.append(detail_input)

    ### 위험
    if detail =="위험 치매, 섬망, 간성호수, 환각(hallucination)등으로 위험행동이 발생했거나 발생할 가능성이 높아 집중적인 관찰을 요하는 상태로 위험행동 예방을 위한 관리를 하고 기록이 있는 경우 *위험행동: 배액관, 주입라인 제거, 자해, 폭행 등 신처적(물리적) 위해가 발생하는 행동":
        detail_input = patient+'_'+date+'_'+'1'
        ADL_danger.append(detail_input)
    elif detail =='위험 해당없음':
        detail_input = patient+'_'+date+'_'+'0'
        ADL_danger.append(detail_input)

    
    ### 전문치료
    if detail =="7개중 1개 이상/1일(마약성주사, 항암치료, 승압제, 항부정맥제, 면역억제제, 항혈전/색전제, 수혈)":
        detail_input = patient+'_'+date+'_'+'1'
        ADL_spe.append(detail_input)
    elif detail =='미시행':
        detail_input = patient+'_'+date+'_'+'0'
        ADL_spe.append(detail_input)
    

    else:
        pass



1302it [00:00, 460725.88it/s]


In [20]:
test['도구명'].value_counts()

ADL_배뇨, 배변          217
ADL_식사섭취            217
ADL_침상 밖으로 이동       217
ADL_체위변경            217
호흡 간호_비침습적 산소 투여    217
위험행동관리              109
전문치료 시행             108
Name: 도구명, dtype: int64

In [21]:
meal_patient = []
meal_date = []
meal_adl = []

for i in ADL_meal:
    patient = i.split('_')[0]
    date = i.split('_')[1]
    adl = i.split('_')[-1]

    meal_patient.append(patient)
    meal_date.append(date)
    meal_adl.append(adl)

adl_meal_df = pd.DataFrame({'등록번호':meal_patient, '평가일자':meal_date, 'ADL_식사섭취':meal_adl})
print(adl_meal_df.shape)
adl_meal_df.head(2)

(217, 3)


,등록번호,평가일자,ADL_식사섭취
0,1393198,2017-07-01,1
1,1393198,2017-07-02,1


In [22]:
outbed_patient = []
outbed_date = []
outbed_adl = []

for i in ADL_outbed:
    patient = i.split('_')[0]
    date = i.split('_')[1]
    adl = i.split('_')[-1]

    outbed_patient.append(patient)
    outbed_date.append(date)
    outbed_adl.append(adl)

adl_outbed_df = pd.DataFrame({'등록번호':outbed_patient, '평가일자':outbed_date, 'ADL_침상 밖으로 이동':outbed_adl})
print(adl_outbed_df.shape)
adl_outbed_df.head(2)

(217, 3)


,등록번호,평가일자,ADL_침상 밖으로 이동
0,1393198,2017-07-01,0
1,1393198,2017-07-02,0


In [23]:
pos_patient = []
pos_date = []
pos_adl = []

for i in ADL_pos:
    patient = i.split('_')[0]
    date = i.split('_')[1]
    adl = i.split('_')[-1]

    pos_patient.append(patient)
    pos_date.append(date)
    pos_adl.append(adl)

adl_pos_df = pd.DataFrame({'등록번호':pos_patient, '평가일자':pos_date, 'ADL_체위변경':pos_adl})
print(adl_pos_df.shape)
adl_pos_df.head(2)

(217, 3)


,등록번호,평가일자,ADL_체위변경
0,1393198,2017-07-01,0
1,1393198,2017-07-02,0


In [24]:
resp_patient = []
resp_date = []
resp_adl = []

for i in ADL_resp:
    patient = i.split('_')[0]
    date = i.split('_')[1]
    adl = i.split('_')[-1]

    resp_patient.append(patient)
    resp_date.append(date)
    resp_adl.append(adl)

adl_resp_df = pd.DataFrame({'등록번호':resp_patient, '평가일자':resp_date, '호흡 간호_비침습적 산소 투여':resp_adl})
print(adl_resp_df.shape)
print(len(adl_resp_df['등록번호'].unique()))
adl_resp_df.head(2)

(217, 3)
60


,등록번호,평가일자,호흡 간호_비침습적 산소 투여
0,1393198,2017-07-01,0
1,1393198,2017-07-02,0


In [25]:
uri_patient = []
uri_date = []
uri_adl = []

for i in ADL_uri:
    patient = i.split('_')[0]
    date = i.split('_')[1]
    adl = i.split('_')[-1]

    uri_patient.append(patient)
    uri_date.append(date)
    uri_adl.append(adl)

adl_uri_df = pd.DataFrame({'등록번호':uri_patient, '평가일자':uri_date, 'ADL_배뇨, 배변':uri_adl})
print(adl_uri_df.shape)
print(len(adl_uri_df['등록번호'].unique()))
adl_uri_df.head(2)

(217, 3)
60


,등록번호,평가일자,"ADL_배뇨, 배변"
0,1393198,2017-07-01,0
1,1393198,2017-07-02,0


In [26]:
danger_patient = []
danger_date = []
danger_adl = []

for i in ADL_danger:
    patient = i.split('_')[0]
    date = i.split('_')[1]
    adl = i.split('_')[-1]

    danger_patient.append(patient)
    danger_date.append(date)
    danger_adl.append(adl)

adl_danger_df = pd.DataFrame({'등록번호':danger_patient, '평가일자':danger_date, 'ADL_위험':danger_adl})
print(adl_danger_df.shape)
print(len(adl_danger_df['등록번호'].unique()))
adl_danger_df.head(2)

(109, 3)
34


,등록번호,평가일자,ADL_위험
0,796422,2019-12-21,0
1,796422,2019-12-22,0


In [27]:
spe_patient = []
spe_date = []
spe_adl = []

for i in ADL_spe:
    patient = i.split('_')[0]
    date = i.split('_')[1]
    adl = i.split('_')[-1]

    spe_patient.append(patient)
    spe_date.append(date)
    spe_adl.append(adl)

adl_spe_df = pd.DataFrame({'등록번호':spe_patient, '평가일자':spe_date, 'ADL_전문':spe_adl})
print(adl_spe_df.shape)
print(len(adl_spe_df['등록번호'].unique()))
adl_spe_df.head(2)

(108, 3)
27


,등록번호,평가일자,ADL_전문
0,1393198,2017-07-01,1
1,1393198,2017-07-02,1


In [28]:
#### special_info_df
adl_danger_df.to_csv('천안_위험행동관리.csv')
adl_spe_df.to_csv('천안_전문치료.csv')

In [29]:
print(len(ADL_uri))
print(len(ADL_resp))
print(len(ADL_pos))
print(len(ADL_outbed))
print(len(ADL_meal))
print(len(ADL_spe))
print(len(ADL_danger))

217
217
217
217
217
108
109


In [30]:
needs['도구명'].value_counts()

ADL_배뇨, 배변          217
ADL_식사섭취            217
ADL_침상 밖으로 이동       217
ADL_체위변경            217
호흡 간호_비침습적 산소 투여    217
위험행동관리              109
전문치료 시행             108
Name: 도구명, dtype: int64

In [31]:
final_adl = adl_uri_df.merge(adl_pos_df, on=['등록번호', '평가일자'], how='inner')
final_adl = final_adl.merge(adl_outbed_df, on=['등록번호', '평가일자'], how='inner')
final_adl = final_adl.merge(adl_meal_df, on=['등록번호', '평가일자'], how='inner')
final_adl = final_adl.merge(adl_resp_df, on=['등록번호', '평가일자'], how='inner')
len(final_adl['등록번호'].unique())

60

In [32]:
final_adl = final_adl.drop_duplicates()
len(final_adl['등록번호'].unique())

60

In [33]:
final_adl

,등록번호,평가일자,"ADL_배뇨, 배변",ADL_체위변경,ADL_침상 밖으로 이동,ADL_식사섭취,호흡 간호_비침습적 산소 투여
0,1393198,2017-07-01,0,0,0,1,0
1,1393198,2017-07-02,0,0,0,1,0
2,1393198,2017-07-05,0,0,0,1,0
3,1393198,2017-07-06,0,0,0,1,0
4,1393198,2017-07-07,0,0,0,1,0
...,...,...,...,...,...,...,...
272,594511,2021-07-10,0,0,0,0,0
273,860045,2021-07-22,0,0,1,0,0
274,860045,2021-07-23,1,0,1,0,0
275,860045,2021-07-24,1,0,1,0,0


In [35]:
final_adl.to_csv('천안_간호필요도.csv')